In [22]:
import os
import sys
import glob
import cv2
import numpy as np
import pandas as pd
import imutils
import operator
import math
import struct
import dlib
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def centroid_histogram(clt):
    # grab the number of different clusters and create a histogram
    # based on the number of pixels assigned to each cluster
    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins = numLabels)

    # normalize the histogram, such that it sums to one
    hist = hist.astype("float")
    hist /= hist.sum()

    # return the histogram
    return hist

def plot_colors(hist, centroids):
    # initialize the bar chart representing the relative frequency
    # of each of the colors
    bar = np.zeros((50, 300, 3), dtype = "uint8")
    startX = 0
    # loop over the percentage of each cluster and the color of
    # each cluster
    for (percent, color) in zip(hist, centroids):
        # plot the relative percentage of each cluster
        endX = startX + (percent * 300)
        cv2.rectangle(bar, (int(startX), 0), (int(endX), 50),color.astype("uint8").tolist(), -1)
        startX = endX

    # return the bar chart
    return bar

ddepth = cv2.CV_16S
scale = 1
delta = 0


f ='/home/nikhil/1000lookz/practice/task/k10.jpg'

f2=os.path.splitext(f)[0]
f2 = os.path.split(f2)[1]


print(f2)


img= cv2.imread(f)
temp=cv2.imread(f)
img2= cv2.imread(f)
img3= cv2.imread(f)
#negate the image

bg_color=img[0][0]

temp=cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)


cv2.imwrite('check.png', temp)

h,w=img.shape[0:2]
img2=cv2.rectangle(img2,(0,0),(w,h),(255,255,255),-1)

# Constants for finding range of skin color in YCrCb and extract the pixels without the skin tone
min_YCrCb = np.array([0,123,77],np.uint8)
max_YCrCb = np.array([255,173,130],np.uint8)



# Convert image to YCrCb
imageYCrCb = cv2.cvtColor(temp,cv2.COLOR_RGB2YCR_CB)

# Find region with skin tone in YCrCb image
skinRegion = cv2.inRange(imageYCrCb,min_YCrCb,max_YCrCb)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
skinRegion = cv2.erode(skinRegion, kernel, iterations = 3)
skinRegion = cv2.dilate(skinRegion, kernel, iterations = 3)

skinRegion = cv2.GaussianBlur(skinRegion, (3, 3), 0)

skinRegion=255-skinRegion

skin = cv2.bitwise_and(img, img, mask = skinRegion)

skin[np.where((skin == [0,0,0]).all(axis = 2))] = bg_color

cv2.imwrite('Foreground.png', skin)

skin = cv2.Canny(skin,100,500)
kernel = np.ones((3,3), np.uint8)
skin = cv2.dilate(skin, kernel, iterations=2)

skin = cv2.Canny(skin,100,500)
skin = cv2.dilate(skin, kernel, iterations=1)
skin = cv2.erode(skin, kernel, iterations=1)

cnts, _ = cv2.findContours(skin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
c = max(cnts, key = cv2.contourArea)


cv2.drawContours(img2, c, -1, (0,0,0), thickness=0)
th, img2 = cv2.threshold(img2, 220, 255, cv2.THRESH_BINARY_INV)
im_floodfill = img2.copy()
h, w = img2.shape[:2]
mask = np.zeros((h+2, w+2), np.uint8)
 
# Floodfill from point (0, 0)
cv2.floodFill(im_floodfill, mask, (0,0), 255);
 
# Invert floodfilled image
im_floodfill_inv = cv2.bitwise_not(im_floodfill)
 
# Combine the two images to get the foreground.
im_out = img2 | im_floodfill_inv

black=im_out[0][0]

im_out[np.where((im_out == black).all(axis = 2))] = [0,0,0]
cv2.imwrite('Foreground.png', im_out)
im_out = cv2.erode(im_out, kernel, iterations=5)


print(img3.shape)
print(im_out.shape)
res =cv2.bitwise_and(src1 = img3, src2 = im_out)

res = cv2.cvtColor(res, cv2.COLOR_RGB2RGBA)
res[np.where((im_out == [0,0,0]).all(axis = 2))] = [255,255,255,0]
cv2.imwrite('Foreground'+f2+'.png', res)
res = res.reshape((res.shape[0] * res.shape[1], 4))


k10
(508, 400, 3)
(508, 400, 3)


In [23]:
res = [x for x in res if x[3] != 0]
print(len(res))

#my system struggles ,that's why i shuffled the pixel array and took 1/3th of the sample,results will be more accuratte with all pixel values.
si=len(res)//4
np.random.shuffle(res)

range_n_clusters = list (range(2,6))

plist=[]
for n_clusters in range_n_clusters:
    clusterer = KMeans (n_clusters=n_clusters)
    preds = clusterer.fit_predict(res)
    centers = clusterer.cluster_centers_
    score = silhouette_score (res, preds,sample_size=si, metric='euclidean')
    plist.append([n_clusters, score])

def Sort(sub_li): 
  
    # reverse = None (Sorts in Descending order) 
    # key is set to sort using second element of  
    # sublist lambda has been used 
    sub_li.sort(key = lambda x: x[1],reverse=True) 
    return sub_li 

print(Sort(plist))  
clt=plist[0][0]

# cluster the pixel intensities
clt = KMeans(n_clusters = 2)
clt.fit(res)

# build a histogram of clusters and then create a figure
# representing the number of pixels labeled to each color
hist = centroid_histogram(clt)
bar = plot_colors(hist, clt.cluster_centers_)
cv2.imwrite('bar'+f2+'.png', bar)

10246
[[2, 0.6637034975186942], [4, 0.5846399540961992], [3, 0.5758442029359464], [5, 0.5437032583905781]]


True